# Necessary Steps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MediaCloud

/content/drive/MyDrive/MediaCloud


In [ ]:
import os
pwd = os.getcwd()
pwd

'/content/drive/MyDrive/MediaCloud'

# Install Libraries

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Hugging Face and Libraries

In [ ]:
pwd

'/content/drive/MyDrive/MediaCloud'

In [ ]:
from huggingface_hub import login

# Use your Hugging Face access token (get it from https://huggingface.co/settings/tokens)
login('ENTER TOKEN HERE')

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
import pandas as pd
import os

# Model

In [ ]:
model = "meta-llama/Llama-3.2-1B-Instruct"
#model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=7000,
    max_new_tokens=7000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False,
    truncation=True
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

<ipython-input-10-9410ce7de563>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


# Output -One by One


In [ ]:
df = pd.read_csv('Output/7.Separate_Clusters/2023/Infrastructure.csv')

In [ ]:
text = ' '.join(df['parsed_summary'].astype(str))

In [ ]:
#text

In [ ]:
template = """
              Using the article summaries below (delimited by triple backticks) related to Infrastructure for the year 2023,
              generate a cohesive, story-like summary that connects key themes, developments, and policy shifts.
              Additionally, explain any technical or financial jargon in the following format: jargon (explanation here in simple terms).
              Do NOT repeat the input text or the instructions. Only return the summary and key takeaways.
              ```{text}```
              SUMMARY:
           """



prompt = PromptTemplate(template=template, input_variables=["text"])

# Assuming `llm` (e.g., HuggingFacePipeline or another LLM) is already defined
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
output = llm_chain.run(text)

Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
# Specify the folder and file name
folder = "Output/8.Cluster_Summaries/2023/"
os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist
file_path = os.path.join(folder, "Infrastructure.txt")


In [ ]:
with open(file_path, "w", encoding="utf-8") as file:
    file.write(output)

print(f"Summary saved to {file_path}")

# Output all at once

In [ ]:
topic = "Tax_Policy"

In [ ]:
# List of years to process
#years = ['2019', '2020', '2021', '2022', '2023', '2024']
years = ['2020', '2021', '2022', '2024']

In [ ]:
# Define the input and output folder paths
input_base_folder = "Farmers_Protest/7.Separate_Clusters"
output_base_folder = "Farmers_Protest/8.Cluster_Summaries"


In [ ]:

template = """
              Using the article summaries below (delimited by triple backticks) related to Healthcare, generate a story-like summary that connects key themes, developments, and policy shifts.
              Additionally, explain any technical or financial jargon within the generated summary in the following format: jargon (explanation of jargon in simple terms).
              Do NOT repeat the input text or the instructions and explain jargons within the summary.
              ```{text}```
              SUMMARY:
           """


prompt = PromptTemplate(template=template, input_variables=["text"])

# Assuming `llm` (e.g., HuggingFacePipeline or another LLM) is already defined
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
# Batch size
batch_size = 20

In [ ]:
# Iterate through each year
for year in years:
    # Input and output paths for the year
    input_file_path = os.path.join(input_base_folder, year, f"{topic}.csv")
    output_folder = os.path.join(output_base_folder, year)
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    output_file_path = os.path.join(output_folder, f"{topic}_summary.txt")

    # Check if the input file exists
    if os.path.exists(input_file_path):
        # Load the CSV file
        df = pd.read_csv(input_file_path)

        # Initialize an empty list to store batch outputs
        all_outputs = []

        # Process in batches of `batch_size`
        for i in range(0, len(df), batch_size):
            # Select a batch of rows
            batch_text = ' '.join(df['parsed_summary'].iloc[i:i + batch_size].astype(str))

            # Generate the summary for the batch
            output = llm_chain.run(batch_text)

            # Append the output to the list
            all_outputs.append(output)

        # Combine all batch outputs into a single string
        final_output = '\n\n'.join(all_outputs)

        # Save the final output to the file
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write(final_output)

        print(f"Summary saved to {output_file_path}")
    else:
        print(f"File not found: {input_file_path}")

Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary saved to Output/8.Cluster_Summaries/2019/Tax_Policy_summary.txt


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


# Output all at once - Except 2019 and 2020

In [ ]:
#topic = "Infrastructure"
topic = "Capital_Markets"

In [ ]:
# List of years to process
years = ['2021', '2022', '2023', '2024']


In [ ]:
# Define the input and output folder paths
input_base_folder = "Output/8.UPDATED_Separate_Clusters"
output_base_folder = "Output/9.Cluster_Summaries"


In [ ]:
# Template for generating the summary
template = """
Write a long summary and key takeaways of the following text delimited by triple backticks.
Do NOT repeat the input text or the instructions.
Only return the summary and key takeaways.
```{text}```
SUMMARY:
"""
prompt = PromptTemplate(template=template, input_variables=["text"])

# Assuming `llm` (e.g., HuggingFacePipeline or another LLM) is already defined
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
# Batch size
batch_size = 10


In [ ]:
# Iterate through each year
for year in years:
    # Input and output paths for the year
    input_file_path = os.path.join(input_base_folder, year, f"{topic}.csv")
    output_folder = os.path.join(output_base_folder, year)
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    output_file_path = os.path.join(output_folder, f"{topic}_summary.txt")

    # Check if the input file exists
    if os.path.exists(input_file_path):
        # Load the CSV file
        df = pd.read_csv(input_file_path)

        # Initialize an empty list to store batch outputs
        all_outputs = []

        # Process in batches of `batch_size`
        for i in range(0, len(df), batch_size):
            # Select a batch of rows
            batch_text = ' '.join(df['parsed_summary'].iloc[i:i + batch_size].astype(str))

            # Generate the summary for the batch
            output = llm_chain.run(batch_text)

            # Append the output to the list
            all_outputs.append(output)

        # Combine all batch outputs into a single string
        final_output = '\n\n'.join(all_outputs)

        # Save the final output to the file
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write(final_output)

        print(f"Summary saved to {output_file_path}")
    else:
        print(f"File not found: {input_file_path}")

Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary saved to Output/9.Cluster_Summaries/2021/Capital_Markets_summary.txt


Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary saved to Output/9.Cluster_Summaries/2022/Capital_Markets_summary.txt
Summary saved to Output/9.Cluster_Summaries/2023/Capital_Markets_summary.txt


Both `max_new_tokens` (=7000) and `max_length`(=7000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary saved to Output/9.Cluster_Summaries/2024/Capital_Markets_summary.txt
